In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:17
🔁 Restarting kernel...


In [1]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [2]:
!mamba install -q openmm


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.0.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████

  Package              Version  Build                Channel                   Size
──────────────────────────────

In [3]:
!mamba install -c conda-forge pdbfixer


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.0.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['pdbfixer']

conda-forge/linux-64                                        Using cache
conda-forge/no

In [4]:
from openmm.testInstallation import main 
main()


OpenMM Version: 7.7
Git Revision: 130124a3f9277b054ec40927360a6ad20c8f5fa6

There are 4 Platforms available:

1 Reference - Successfully computed forces
2 CPU - Successfully computed forces
3 CUDA - Successfully computed forces
4 OpenCL - Successfully computed forces

Median difference in forces between platforms:

Reference vs. CPU: 6.30202e-06
Reference vs. CUDA: 6.73411e-06
CPU vs. CUDA: 7.44584e-07
Reference vs. OpenCL: 6.76294e-06
CPU vs. OpenCL: 8.09318e-07
CUDA vs. OpenCL: 2.64451e-07

All differences are within tolerance.


In [1]:
import time
import numpy as np
from openmm import app
import openmm as mm
from openmm import unit as u
import pdbfixer
from sys import stdout

In [7]:
STRIDE = 1000
simulations_steps = 6000000
temperature = 300 * u.kelvin
platform_name = 'CPU' 
real_time_STRIDE = 100000
friction_coeff = 1.0/u.picoseconds
time_step = 2.0*u.femtoseconds 
pressure = 1 * u.bar

In [5]:
# Loading the file into the notebook.
pdb_file = './peptide_model_water.pdb'
fixer = pdbfixer.PDBFixer(pdb_file)

In [8]:
# Defining the forcfield and the model
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
model = app.Modeller(fixer.topology, fixer.positions)
model.addSolvent(forcefield,
             padding=1.*u.nanometer,
             #boxSize=mm.Vec3(3.0, 3.0, 3.0) * u.nanometers ,
             model='tip3p', 
             ionicStrength=0.1 * u.millimolar, 
             positiveIon='Na+',
             negativeIon='Cl-',
             neutralize=True )

app.PDBFile.writeFile(model.topology,model.positions, open('peptide_model_water.pdb', 'w'))###add

system = forcefield.createSystem(model.topology, 
                                 nonbondedMethod=app.PME, 
                                 nonbondedCutoff=1.2*u.nanometers, #we did 1.0
                                 constraints=app.HBonds, 
                                 rigidWater=True, 
                                 ewaldErrorTolerance=0.0005)

system.addForce(mm.MonteCarloBarostat(pressure, temperature))###add

integrator = mm.LangevinIntegrator(temperature,
                                   friction_coeff,
                                   time_step)
integrator.setConstraintTolerance(0.00001)

platform = mm.Platform.getPlatformByName(platform_name)
simulation = app.Simulation(model.topology, system, integrator, platform)
simulation.context.setPositions(model.positions)
simulation.context.getState(getVelocities=True, getForces=True).getForces(asNumpy=True)

#start = time.time()###delete
#simulation.reporters.append(mm.app.pdbreporter.PDBReporter('data/output.pdb', STRIDE))###delete
simulation.reporters.append(mm.app.dcdreporter.DCDReporter('output.dcd', STRIDE))###add
simulation.reporters.append(mm.app.statedatareporter.StateDataReporter('mm.log', STRIDE, step=True, potentialEnergy=True, temperature=True, time=True))
simulation.reporters.append(mm.app.statedatareporter.StateDataReporter(stdout, real_time_STRIDE, step=True, potentialEnergy=True, temperature=True, speed=True, elapsedTime=True))
simulation.minimizeEnergy()
simulation.step(simulations_steps)
#end = time.time()

#print(f"Elapsed real time: {end-start} s")###delete
#print(f'I could do {(60*60)/(end-start)*simulations_steps} simulation steps per hour with {simulations_steps/STRIDE} frames saved.')###delete